In [5]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import joblib

In [8]:
data = pd.read_csv('appl_score_sample.csv', delimiter=';')

x = data.drop(columns=['CLIENT_ID', 'Target'])
y = data['Target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

cat_features = ['SEX', 'CHILD_FLAG', 'REALTY_FLAG', 'ACCOUNTS_FLAG', 'E_MAIL_FLAG', 'DCI', 'FOREIGN_PASSPORT_FLAG', 'EDUCATION']


model = XGBClassifier(
        iterations=100, 
        depth=6, 
        learning_rate=0.1, 
        loss_function='Logloss', 
        cat_features=cat_features)
    
model.fit(x_train, y_train)

model.save_model('catboost_model.cbm')
model.predict(x_test)

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:03:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "cat_features", "depth", "iterations", "loss_function" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:03:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [14]:


def load_and_preprocess_data(file_path):

    data = pd.read_csv(file_path, delimiter=';')

    x = data.drop(columns=['CLIENT_ID', 'Target'])
    y = data['Target']

    cat_features = ['SEX', 'CHILD_FLAG', 'REALTY_FLAG', 'ACCOUNTS_FLAG', 'E_MAIL_FLAG', 'DCI', 'FOREIGN_PASSPORT_FLAG', 'EDUCATION']
    return x, y, cat_features

def objective(params):
    clf = XGBClassifier(**params, cat_features=cat_features, verbose=0)
    clf.fit(x_train, y_train)
    loss = -clf.score(x_test, y_test)
    return {'loss': loss, 'status': STATUS_OK}

if __name__ == "__main__":
    c, y, cat_features = load_and_preprocess_data('appl_score_sample.csv')
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    param_space = {
        'iterations': hp.choice('iterations', range(50, 500)),
        'depth': hp.choice('depth', range(3, 10)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
        'border_count': hp.choice('border_count', range(32, 255)),
    }


    trials = Trials()
    best_params = fmin(
        fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=100,
        trials=trials
    )

    print(f"Best parameters: {best_params}")

    best_params['iterations'] = range(50, 500)[best_params['iterations']]
    best_params['depth'] = range(3, 10)[best_params['depth']]
    best_params['border_count'] = range(32, 255)[best_params['border_count']]

    final_model = XGBClassifier(**best_params, cat_features=cat_features)
    final_model.fit(x_train, y_train)

    final_model.save_model('catboost_model_v1.cbm')


  1%|          | 1/100 [00:00<00:14,  6.69trial/s, best loss: -0.605080831408776]

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)



  4%|▍         | 4/100 [00:00<00:12,  7.57trial/s, best loss: -0.628175519630485]

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▌         | 5/100 [00:00<00:22,  4.24trial/s, best loss: -0.628175519630485]

...

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)



 99%|█████████▉| 99/100 [00:12<00:00,  6.32trial/s, best loss: -0.651270207852194]

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)

c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)



100%|██████████| 100/100 [00:12<00:00,  7.76trial/s, best loss: -0.651270207852194]
Best parameters: {'border_count': np.int64(147), 'depth': np.int64(4), 'iterations': np.int64(10), 'l2_leaf_reg': np.float64(6.837205168985412), 'learning_rate': np.float64(0.013748121731017806)}


c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "border_count", "cat_features", "depth", "iterations", "l2_leaf_reg" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\xiaomi\Desktop\model2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:11:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
